### Imports

In [178]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### 2020 Census Population data

In [78]:
url = 'https://www.census.gov/quickfacts/quickfacts/geo/dashboard/US/POP010220'
res = requests.get(url)

In [79]:
soup = BeautifulSoup(res.content)

In [80]:
table = soup.find(attrs={'class':'qf-graph-scroll'})
# soup.find_all(attrs={'class':'qf-positive'})
tbody = table.find_all(attrs={'class':"qf-graph-geo"})
tr = tbody[1]
state = tr.find('a').attrs['data-title']
tr.find(attrs={'class':'qf-positive'}).attrs['data-value']

'29145505'

In [81]:
pop_2020 = []
for tr in tbody:
    state = tr.find('a').attrs['data-title']
    pop = tr.find(attrs={'class':'qf-positive'}).attrs['data-value']
    
    state_pop = {'State': state, '2020 Population': pop}
    pop_2020.append(state_pop)

In [82]:
pop_2020 = pd.DataFrame(pop_2020)

### 2010 Census Population Data

In [83]:
url2 = 'https://www.census.gov/quickfacts/quickfacts/geo/dashboard/US/POP010210'
res2 = requests.get(url2)

In [84]:
soup2 = BeautifulSoup(res2.content)
table2 = soup2.find(attrs={'class':'qf-graph-scroll'})
tbody2 = table2.find_all(attrs={'class':"qf-graph-geo"})

In [85]:
pop_2010 = []
for tr in tbody2:
    state = tr.find('a').attrs['data-title']
    pop = tr.find(attrs={'class':'qf-positive'}).attrs['data-value']
    
    state_pop = {'State': state, '2010 Population': pop}
    pop_2010.append(state_pop)

pop_2010 = pd.DataFrame(pop_2010)

In [86]:
population = pd.merge(pop_2010, pop_2020)
population.head()

,State,2010 Population,2020 Population
0,California,37253956,39538223
1,Texas,25145561,29145505
2,New York,19378102,20201249
3,Florida,18801310,21538187
4,Illinois,12830632,12812508


In [87]:
population.to_csv('Data/Population_data_2010_&_2020.csv')

### Excess Deaths Data

In [88]:
deaths = pd.read_csv('Ignore/Excess_Deaths_Associated_with_COVID-19.csv')
deaths.head()

,Week Ending Date,State,Observed Number,Upper Bound Threshold,Exceeds Threshold,Average Expected Count,Excess Estimate,Total Excess Estimate,Percent Excess Estimate,Year,Type,Outcome,Suppress,Note
0,2017-01-07,Alabama,1121.0,1136,False,1059,62,29601,5.852700,2017,Predicted (weighted),All causes,NaN,NaN
1,2017-01-14,Alabama,1130.0,1140,False,1067,63,29601,5.906102,2017,Predicted (weighted),All causes,NaN,NaN
2,2017-01-21,Alabama,1048.0,1142,False,1071,0,29601,0.000000,2017,Predicted (weighted),All causes,NaN,NaN
3,2017-01-28,Alabama,1026.0,1142,False,1070,0,29601,0.000000,2017,Predicted (weighted),All causes,NaN,NaN
4,2017-02-04,Alabama,1036.0,1142,False,1068,0,29601,0.000000,2017,Predicted (weighted),All causes,NaN,NaN


In [89]:
# getting sum of each states excess deaths 
def excess_deaths(deaths, start_year=2017, end_year=2022):
    state_totals = {state: [0] * (end_year - start_year + 1) for state in 
                    deaths['State'].unique()}
    
    for index, row in deaths.iterrows():
        state = row['State']
        year = row['Year']
        excess_estimate = row['Excess Estimate']
        
        if start_year <= year <= end_year:
            state_totals[state][year - start_year] += excess_estimate

    result = pd.DataFrame(state_totals).T.reset_index()
    result.columns = ['State'] + [f'Exc_deaths_{year}' for year in 
                                  range(start_year, end_year + 1)]
    
    return result


excess_deaths = excess_deaths(deaths)
excess_deaths.head()


,State,Exc_deaths_2017,Exc_deaths_2018,Exc_deaths_2019,Exc_deaths_2020,Exc_deaths_2021,Exc_deaths_2022
0,Alabama,2649,4062,945,20726,29283,14805
1,Alaska,417,492,639,1422,3483,1783
2,Arizona,3522,4281,1785,30955,40347,20653
3,Arkansas,3054,2199,1434,11285,15391,8969
4,California,17241,13434,1029,93875,133206,89195


In [90]:
# Getting count for each year and each state where they exceed their threshold
def count_exceeds_threshold(deaths, start_year=2017, end_year=2022):
    state_counts = {state: [0] * (end_year - start_year + 1) for state in 
                    deaths['State'].unique()}
    
    for index, row in deaths.iterrows():
        state = row['State']
        year = row['Year']
        exceeds_threshold = row['Exceeds Threshold']
        
        if start_year <= year <= end_year and exceeds_threshold:
            state_counts[state][year - start_year] += 1

    result = pd.DataFrame(state_counts).T.reset_index()
    result.columns = ['State'] + [f'Exc_count_{year}' for year in 
                                  range(start_year, end_year + 1)]
    
    return result
exceeds_threshold = count_exceeds_threshold(deaths)
exceeds_threshold.head()

,State,Exc_count_2017,Exc_count_2018,Exc_count_2019,Exc_count_2020,Exc_count_2021,Exc_count_2022
0,Alabama,3,15,0,99,106,87
1,Alaska,6,6,3,34,75,42
2,Arizona,9,15,0,89,114,77
3,Arkansas,12,12,0,66,93,64
4,California,18,12,0,94,104,106


In [124]:
# Merging the data
finaldeaths = pd.merge(excess_deaths, exceeds_threshold, how='left')
finaldeaths.rename(columns={'State':'Location'}, inplace=True)
finaldeaths.head()

,Location,Exc_deaths_2017,Exc_deaths_2018,Exc_deaths_2019,Exc_deaths_2020,Exc_deaths_2021,Exc_deaths_2022,Exc_count_2017,Exc_count_2018,Exc_count_2019,Exc_count_2020,Exc_count_2021,Exc_count_2022
0,Alabama,2649,4062,945,20726,29283,14805,3,15,0,99,106,87
1,Alaska,417,492,639,1422,3483,1783,6,6,3,34,75,42
2,Arizona,3522,4281,1785,30955,40347,20653,9,15,0,89,114,77
3,Arkansas,3054,2199,1434,11285,15391,8969,12,12,0,66,93,64
4,California,17241,13434,1029,93875,133206,89195,18,12,0,94,104,106


In [92]:
finaldeaths.to_csv('Data/Excess_deaths&Exceeds_Threshold_data.csv')

### Population Density Data

In [93]:
url3 = 'https://wisevoter.com/state-rankings/population-density-by-state/'
res3 = requests.get(url3)
soup3 = BeautifulSoup(res3.content)

In [94]:
table3 = soup3.find('table', attrs={'id': 'shdb-on-page-table'})
tbody = table3.find('tbody')
trs = tbody.find_all('tr')

AttributeError: 'NoneType' object has no attribute 'find'

In [ ]:
pop_density = []
for tr in trs:
    state = tr.find(attrs={'class':'shdb-on-page-table-body-Geo'}).text
    density = tr.find(attrs={'class':'shdb-on-page-table-body-Data'}).text.split()[0]
    
    state_density = {'State':state, 'Population Density per mi²': density}
    pop_density.append(state_density)
Pop_density = pd.DataFrame(pop_density)
Pop_density.head()

In [ ]:
Pop_density.to_csv('Data/Population_Density_data.csv')

### Mask county data

In [200]:
county_mask = pd.read_csv('Ignore/U.S._State_and_Territorial_Public_Mask_Mandates_From_April_10__2020_through_August_15__2021_by_County_by_Day.csv')
county_mask

In [202]:
# Put my code into Chatgpt who helped simplify to speed up run time
def mandate_length(data):
    county_counts = {}
    for index, row in data.iterrows():
        county_name = row['County_Name']
        mask_required = row['Face_Masks_Required_in_Public']
        state = row['State_Tribe_Territory']
        
        if mask_required == 'Yes':
            if state in county_counts:
                if county_name in county_counts[state]:
                    county_counts[state][county_name]['Count'] += 1
                else:
                    county_counts[state][county_name] = {'State': state, 'Count': 1}
            else:
                county_counts[state] = {county_name: {'State': state, 'Count': 1}}

    county_counts_list = [{'State': state, 'County': county_name, 'Count': county_counts[state][county_name]['Count']} for state in county_counts for county_name in county_counts[state]]

    return pd.DataFrame(county_counts_list)

county_counts_df = mandate_length(county_mask)

In [207]:
county_counts_df.head()

,State,County,Count
0,AL,Autauga County,267
1,AL,Baldwin County,267
2,AL,Barbour County,267
3,AL,Bibb County,267
4,AL,Blount County,267


In [204]:
county_counts_df.shape

(2423, 3)

In [205]:
county_counts_df['State'].value_counts()

TX    254
VA    133
KY    120
KS    105
IL    102
NC    100
IA     99
IN     92
OH     88
MN     87
MI     83
MS     82
PR     78
AR     75
WI     72
PA     67
AL     67
LA     64
CO     64
NY     62
CA     58
MT     56
WV     55
ND     53
WA     39
OR     36
NM     33
UT     29
MD     24
WY     23
NJ     21
NV     17
ME     16
MA     14
VT     14
NH     10
CT      8
RI      5
HI      5
AS      5
DE      3
VI      3
GU      1
DC      1
Name: State, dtype: int64

In [206]:
county_counts_df.to_csv('Data/county_mask_mandata.csv', index=False)

### Merging the data

In [183]:
def merge_dataframes():
    employ = pd.read_csv('Data/Cleaned/cleaned_employment.csv')
    income = pd.read_csv('Data/Cleaned/cleaned_income.csv')
    life = pd.read_csv('Data/Cleaned/cleaned_Life_Expentency.csv')
    insur = pd.read_csv('Data/Cleaned/cleaned_insur.csv')
    pop_dense = pd.read_csv('Data/Cleaned/cleaned_pop_dense.csv')
    pop_size = pd.read_csv('Data/Cleaned/cleaned_pop_size.csv')
    pre_cond = pd.read_csv('Data/Cleaned/cleaned_pre_condtions.csv')
    total_phys = pd.read_csv('Data/Cleaned/cleaned_total_physician.csv')
    
    merged_df = employ.merge(income, on='Location').merge(life, on='Location').merge(insur, on='Location') \
        .merge(pop_dense, on='Location').merge(pop_size, on='Location').merge(pre_cond, on='Location') \
        .merge(total_phys, on='Location').merge(finaldeaths, on='Location')

    return merged_df
merged_data = merge_dataframes()

In [184]:
merged_data.drop(columns=['Unnamed: 0'], inplace=True)
merged_data.head()


,Location,Employment_2020,Employment_2021,Employment_2022,Inc_Per_Cap_2020,Inc_Per_Cap_2021,Inc_Per_CAp_2022,Life_Exp_2020,Life_Exp_2019,Life_Exp_2018,Employer_2019,Non-Group_2019,Medicaid_2019,Medicare_2019,Military_2019,Uninsured_2019,Employer_2021,Non-Group_2021,Medicaid_2021,Medicare_2021,Military_2021,Uninsured_2021,Population Density per mi²,2010 Population,2020 Population,flu_vaccination_rate_2019,asthma_prevalence,cardiac_mortality_rate,high_bp_prevalence,copd_prevalence,kidney_disease_prevalence,diabetes_prevalence,Physicians,Physicians Rate,Active MO,Active MO Rate,Active DO,Active DO Rate,Exc_deaths_2017,Exc_deaths_2018,Exc_deaths_2019,Exc_deaths_2020,Exc_deaths_2021,Exc_deaths_2022,Exc_count_2017,Exc_count_2018,Exc_count_2019,Exc_count_2020,Exc_count_2021,Exc_count_2022
0,Alabama,2671005,2769464,2869931,45887,50059,50916,73.2,75.2,75.1,0.472,0.055,0.195,0.160,0.021,0.097,0.466,0.060,0.192,0.162,0.021,0.100,100.65,4779736,5024279,39.5,9.4,289.3,39.4,9.3,3.1,12.2,"10,983",224.0,"10,389",211.9,594,12.1,2649,4062,945,20726,29283,14805,3,15,0,99,106,87
1,Alaska,430840,443047,457687,61898,65662,68635,76.6,77.7,78.0,0.484,0.035,0.213,0.100,0.053,0.115,0.433,0.041,0.257,0.107,0.053,0.108,1.3,710231,733391,37.0,9.7,178.5,32.8,4.6,1.8,7.1,"2,101",287.2,"1,825",249.5,276,37.7,417,492,639,1422,3483,1783,6,6,3,34,75,42
2,Arizona,3920033,4086802,4287595,52133,56420,58442,76.3,78.8,78.7,0.451,0.052,0.210,0.161,0.015,0.111,0.450,0.054,0.213,0.162,0.015,0.106,64.96,6392017,7151502,37.1,9.8,181.8,29.9,6.0,3.6,9.8,"18,343",252.0,"15,989",219.7,"2,353",32.3,3522,4281,1785,30955,40347,20653,9,15,0,89,114,77
3,Arkansas,1639829,1686444,1755536,47147,51636,52618,73.8,75.7,75.6,0.420,0.054,0.262,0.159,0.014,0.091,0.411,0.056,0.270,0.156,0.015,0.092,58.43,2915918,3011524,40.1,9.3,284.3,38.2,9.7,3.7,12.2,"6,500",215.4,"6,132",203.2,368,12.2,3054,2199,1434,11285,15391,8969,12,12,0,66,93,64
4,California,23154091,23934549,25300974,70061,76991,77036,79.0,80.9,80.8,0.480,0.066,0.253,0.114,0.009,0.078,0.470,0.070,0.265,0.117,0.008,0.070,258.21,37253956,39538223,40.7,7.8,192.5,26.6,4.2,2.8,9.4,"113,718",287.8,"107,383",271.8,"6,333",16.0,17241,13434,1029,93875,133206,89195,18,12,0,94,104,106


In [185]:
merged_data['Population Density per mi²'] = merged_data['Population Density per mi²'].str.replace(',', '', regex=True).astype(float)
merged_data['Physicians'] = merged_data['Physicians'].str.replace(',', '', regex=True).astype(float)
merged_data['Active DO'] = merged_data['Active DO'].str.replace(',', '', regex=True).astype(float)
merged_data['Active MO'] = merged_data['Active MO'].str.replace(',', '', regex=True).astype(float)


In [186]:
merged_data.dtypes

Location                       object
Employment_2020                 int64
Employment_2021                 int64
Employment_2022                 int64
Inc_Per_Cap_2020                int64
Inc_Per_Cap_2021                int64
Inc_Per_CAp_2022                int64
Life_Exp_2020                 float64
Life_Exp_2019                 float64
Life_Exp_2018                 float64
Employer_2019                 float64
Non-Group_2019                float64
Medicaid_2019                 float64
Medicare_2019                 float64
Military_2019                 float64
Uninsured_2019                float64
Employer_2021                 float64
Non-Group_2021                float64
Medicaid_2021                 float64
Medicare_2021                 float64
Military_2021                 float64
Uninsured_2021                float64
Population Density per mi²    float64
2010 Population                 int64
2020 Population                 int64
flu_vaccination_rate_2019     float64
asthma_preva

In [187]:
merged_data.isnull().sum()

Location                      0
Employment_2020               0
Employment_2021               0
Employment_2022               0
Inc_Per_Cap_2020              0
Inc_Per_Cap_2021              0
Inc_Per_CAp_2022              0
Life_Exp_2020                 0
Life_Exp_2019                 0
Life_Exp_2018                 0
Employer_2019                 0
Non-Group_2019                0
Medicaid_2019                 0
Medicare_2019                 0
Military_2019                 0
Uninsured_2019                0
Employer_2021                 0
Non-Group_2021                0
Medicaid_2021                 0
Medicare_2021                 0
Military_2021                 0
Uninsured_2021                0
Population Density per mi²    0
2010 Population               0
2020 Population               0
flu_vaccination_rate_2019     1
asthma_prevalence             1
cardiac_mortality_rate        0
high_bp_prevalence            1
copd_prevalence               1
kidney_disease_prevalence     1
diabetes

In [161]:
merged_data.to_csv('Data/Merged_state_data.csv', index=False)

### Modeling 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LogisticRegression

In [191]:
merged_data.dropna(inplace=True)

In [192]:
X = merged_data.drop(columns=['Location','Exc_deaths_2017', 'Exc_deaths_2018','Exc_deaths_2019',
                              'Exc_deaths_2020', 'Exc_deaths_2021','Exc_deaths_2022',
                             'Exc_count_2017', 'Exc_count_2018', 'Exc_count_2019', 
                             'Exc_count_2020', 'Exc_count_2021', 'Exc_count_2022'])
y = merged_data[['Exc_deaths_2017', 'Exc_deaths_2018','Exc_deaths_2019','Exc_deaths_2020',
                'Exc_deaths_2021','Exc_deaths_2022']]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [193]:
lr1 = LinearRegression()
lr1.fit(X_train, y_train)

LinearRegression()

In [194]:
lr1.score(X_train, y_train)

1.0

In [195]:
lr1.score(X_test, y_test)

-2.3508622476820444